## Training

In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")
model.train(
    data="dataset/prepared_dataset/data.yaml", 
    epochs=400, 
    imgsz=640,
    batch=16, 
    lr0=0.01,
    momentum=0.937
)

metrics = model.val()
print(metrics)

## Test

In [1]:
from ultralytics import YOLO
model = YOLO("model.pt")
results = model(["test.jpeg"], conf=0.5)
results[0].show()


0: 640x480 1 lays-large, 1 lays-small, 1 milo-180ml, 1 slice-200ml, 64.0ms
Speed: 20.0ms preprocess, 64.0ms inference, 252.2ms postprocess per image at shape (1, 3, 640, 480)


In [ ]:
import cv2
from ultralytics import YOLO

model = YOLO("model.pt")
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model.track(frame, persist=True, conf=0.85)
        annotated_frame = results[0].plot()
        cv2.imshow("YOLOv8 Tracking", annotated_frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
from ultralytics import YOLO

# Load the model
model = YOLO("model.pt")

# Open the video capture
cap = cv2.VideoCapture(0)

# Define product price mapping
product_prices = {
    "lays-large": 100,  # Price in Rs
    "lays-small": 50,   # Price in Rs
    "milo-180ml": 80,   # Price in Rs
    "slice-200ml": 60   # Price in Rs
}

# Initialize the counter and a set to track processed object IDs
total_price = 0
processed_object_ids = set()

# Define the line coordinates (you can adjust these as needed)
line_x = 320  # X-coordinate of the line

def reset_counter():
    global total_price, processed_object_ids
    total_price = 0
    processed_object_ids.clear()

def is_box_intersecting_line(box, line_x):
    x1, y1, x2, y2 = box.xyxy[0]
    return x1 <= line_x <= x2 or x2 <= line_x <= x1

while cap.isOpened():
    success, frame = cap.read()

    if success:
        # Perform object detection and tracking
        results = model.track(frame, persist=True, conf=0.85)
        annotated_frame = results[0].plot()

        # Draw the line on the frame
        cv2.line(annotated_frame, (line_x, 0), (line_x, frame.shape[0]), (0, 255, 0), 2)

        # Extract detected objects and update the counter
        if results[0].boxes is not None:
            for box in results[0].boxes:
                class_id = results[0].names[box.cls.item()]
                object_id = box.id.item() if box.id is not None else None

                if class_id in product_prices and object_id not in processed_object_ids:
                    if is_box_intersecting_line(box, line_x):
                        total_price += product_prices[class_id]
                        processed_object_ids.add(object_id)

        # Display the total price on the frame
        cv2.putText(annotated_frame, f"Total Price: Rs {total_price}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the annotated frame
        cv2.imshow("YOLOv8 Tracking", annotated_frame)

        # Check for key presses
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
        elif key == ord("r"):
            reset_counter()
    else:
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()